## 데이터 전처리

### (1) 가격 + 수급안정화지수 + 재고량

In [1]:
import pandas as pd
from scipy import stats
import numpy as np

In [2]:
# 데이터 불러오기
price_data = pd.read_excel('니켈+가격_monthly.xls', skiprows=9)
stability_index_data = pd.read_excel('니켈+수급안정화지수2024-06-10.xls',skiprows=2)


# 필요 없는 데이터 삭제
price_data.drop(price_data.columns[[0,6,7,8,9]], axis=1, inplace=True)
stability_index_data.drop(stability_index_data.columns[0], axis=1, inplace=True)


# 기준일 컬럼 데이터 날짜로 변경
price_data['기준일'] = pd.to_datetime(price_data['기준일'])
stability_index_data['기준일'] = stability_index_data['기준일'].astype(str).apply(lambda x: f"{x[:4]}-{x[4:]}")
stability_index_data['기준일'] = pd.to_datetime(stability_index_data['기준일'])

# 기간 설정
price_data = price_data[(price_data['기준일']>='2017-01-01') & (price_data['기준일']<'2024-01-01')]


# # 데이터 병합
df_nickel = pd.merge(price_data, stability_index_data, on='기준일')



In [3]:
df_nickel

,기준일,기준가격,등락가,등락비율,재고량,수급안정화지수,전월대비 등락가,전월대비 등락율
0,2017-01-01,9984.29,-1028.96,-9.34,374047,NaN,0.00,0.00
1,2017-02-01,10619.50,635.21,6.36,381745,78.24,0.00,0.00
2,2017-03-01,10230.43,-389.07,-3.66,381883,74.56,-3.68,-4.70
3,2017-04-01,9668.61,-561.82,-5.49,375819,77.10,2.54,3.41
4,2017-05-01,9154.29,-514.32,-5.32,380589,80.06,2.96,3.84
...,...,...,...,...,...,...,...,...
79,2023-08-01,20497.73,-400.60,-1.92,37074,28.00,-1.55,-5.26
80,2023-09-01,19629.05,-868.68,-4.24,39516,30.97,2.97,10.62
81,2023-10-01,18255.23,-1373.82,-7.00,43572,31.98,1.01,3.26
82,2023-11-01,16979.77,-1275.46,-6.99,43515,41.74,9.76,30.53


In [4]:
df_nickel.drop(df_nickel.columns[[2,3,6,7]], axis=1, inplace=True)

In [5]:
df_nickel

,기준일,기준가격,재고량,수급안정화지수
0,2017-01-01,9984.29,374047,NaN
1,2017-02-01,10619.50,381745,78.24
2,2017-03-01,10230.43,381883,74.56
3,2017-04-01,9668.61,375819,77.10
4,2017-05-01,9154.29,380589,80.06
...,...,...,...,...
79,2023-08-01,20497.73,37074,28.00
80,2023-09-01,19629.05,39516,30.97
81,2023-10-01,18255.23,43572,31.98
82,2023-11-01,16979.77,43515,41.74


In [6]:
# 원/달러 환율 데이터
er_m = pd.read_csv('exchangerate_monthly.csv', encoding='euc-kr')
er_m = er_m[er_m.columns[[0,4]]]

er_m.rename(columns={'Date':'기준일', 'Close':'환율'},inplace=True)
er_m['기준일'] = pd.to_datetime(er_m['기준일'])

In [7]:
er_m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   기준일     84 non-null     datetime64[ns]
 1   환율      84 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 1.4 KB


In [36]:
df_nickel.head(10)

,기준일,기준가격,재고량,수급안정화지수,환율,생산량,소비량
0,2017-01-01,9984.29,374047,76.63,1169.209961,172.16,174.21
1,2017-02-01,10619.50,381745,78.24,1133.939941,172.16,174.21
2,2017-03-01,10230.43,381883,74.56,1117.020020,172.16,174.21
3,2017-04-01,9668.61,375819,77.10,1135.640015,172.16,174.21
4,2017-05-01,9154.29,380589,80.06,1123.900024,172.16,174.21
5,2017-06-01,8930.68,377252,82.15,1146.140015,172.16,174.21
6,2017-07-01,9481.67,374066,82.86,1124.050049,172.16,174.21
7,2017-08-01,10852.95,380702,79.39,1123.329956,172.16,174.21
8,2017-09-01,11233.57,383687,71.55,0.115380,172.16,174.21
9,2017-10-01,11325.00,385273,68.84,1123.449951,172.16,174.21


In [35]:
df_nickel[df_nickel['환율']<5]

,기준일,기준가격,재고량,수급안정화지수,환율,생산량,소비량
8,2017-09-01,11233.57,383687,71.55,0.11538,172.16,174.21


In [37]:
# 날짜가 '2017-09-01'인 행의 환율 값을 1123.40으로 변경
df_nickel.loc[df_nickel['기준일'] == '2017-09-01', '환율'] = 1123.40

In [38]:
df_nickel.head(10)

,기준일,기준가격,재고량,수급안정화지수,환율,생산량,소비량
0,2017-01-01,9984.29,374047,76.63,1169.209961,172.16,174.21
1,2017-02-01,10619.50,381745,78.24,1133.939941,172.16,174.21
2,2017-03-01,10230.43,381883,74.56,1117.020020,172.16,174.21
3,2017-04-01,9668.61,375819,77.10,1135.640015,172.16,174.21
4,2017-05-01,9154.29,380589,80.06,1123.900024,172.16,174.21
5,2017-06-01,8930.68,377252,82.15,1146.140015,172.16,174.21
6,2017-07-01,9481.67,374066,82.86,1124.050049,172.16,174.21
7,2017-08-01,10852.95,380702,79.39,1123.329956,172.16,174.21
8,2017-09-01,11233.57,383687,71.55,1123.400000,172.16,174.21
9,2017-10-01,11325.00,385273,68.84,1123.449951,172.16,174.21


In [9]:
# 첫 번째 NaN => 2017년 1월 1일 기준가격과 유사한 3개월치(17년 2월-4월) 수급안정화지수의 평균값으로 대체
mean_value = df_nickel.loc[1:3, '수급안정화지수'].mean()
df_nickel.loc[0, '수급안정화지수'] = round(mean_value, 2)

In [10]:
df_nickel

,기준일,기준가격,재고량,수급안정화지수
0,2017-01-01,9984.29,374047,76.63
1,2017-02-01,10619.50,381745,78.24
2,2017-03-01,10230.43,381883,74.56
3,2017-04-01,9668.61,375819,77.10
4,2017-05-01,9154.29,380589,80.06
...,...,...,...,...
79,2023-08-01,20497.73,37074,28.00
80,2023-09-01,19629.05,39516,30.97
81,2023-10-01,18255.23,43572,31.98
82,2023-11-01,16979.77,43515,41.74


In [11]:
df_nickel = pd.merge(df_nickel, er_m, how='left', on='기준일')

In [12]:
df_nickel

,기준일,기준가격,재고량,수급안정화지수,환율
0,2017-01-01,9984.29,374047,76.63,1169.209961
1,2017-02-01,10619.50,381745,78.24,1133.939941
2,2017-03-01,10230.43,381883,74.56,1117.020020
3,2017-04-01,9668.61,375819,77.10,1135.640015
4,2017-05-01,9154.29,380589,80.06,1123.900024
...,...,...,...,...,...
79,2023-08-01,20497.73,37074,28.00,1323.030029
80,2023-09-01,19629.05,39516,30.97,1351.930054
81,2023-10-01,18255.23,43572,31.98,1348.069946
82,2023-11-01,16979.77,43515,41.74,1288.359985


### (2) 생산량 + 소비현황

In [13]:
# 데이터 불러오기
production_data = pd.read_csv('광종별국가별생산량.csv', encoding='euc-kr')
consumption_data = pd.read_csv('광종별 소비현황_20231231.csv', encoding='euc-kr')

# 필요없는 데이터 제거
production_data = production_data[(production_data['광종']=='니켈') & (production_data['품목']=='refined')]
# 소비현황 데이터에 refined만 있으므로 생산량 데이터에서 nickel refined만 가져오기
consumption_data = consumption_data[consumption_data['광종']=='니켈']


In [14]:
# 필요없는 컬럼 제거
production_data.drop(production_data.columns[[0,1,2,3,4]], axis=1, inplace=True)

# 행열전환
production_data = production_data.T

# 연도별 합계 계산해서 데이터프레임 생성
production_data= production_data.sum(axis=1).to_frame(name='생산량')

# 인덱스 이름 연도(숫자)만 나오게 변경
production_data.index = production_data.index.str.replace(' 생산량', '')

In [15]:
# 인덱스 컬럼으로 변경, '연도'로 이름 변경
production_data.reset_index(inplace=True)
production_data.columns = ['연도', '생산량']

In [16]:
# 필요없는 컬럼 제거
consumption_data.drop(consumption_data.columns[[0,1,2,3,4]], axis=1, inplace=True)

# 행열전환
consumption_data = consumption_data.T

# 연도별 합계 계산해서 데이터프레임 생성
consumption_data= consumption_data.sum(axis=1).to_frame(name='소비량')

# 인덱스 이름 연도(숫자)만 나오게 변경
consumption_data.index = consumption_data.index.str.replace(' 소비량', '')

In [17]:
# 인덱스 컬럼으로 변경, '연도'로 이름 변경
consumption_data.reset_index(inplace=True)
consumption_data.columns = ['연도', '소비량']

In [18]:
consumption_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   연도      13 non-null     object 
 1   소비량     13 non-null     float64
dtypes: float64(1), object(1)
memory usage: 336.0+ bytes


In [19]:
production_consumption = pd.merge(production_data, consumption_data, on='연도')

In [20]:
production_consumption['연도'] = production_consumption['연도'].astype('int')

In [21]:
# 2017년부터 2023년 데이터만 필요
production_consumption = production_consumption[production_consumption['연도']>=2017]
production_consumption.reset_index(drop=True, inplace=True)

In [22]:
production_consumption

,연도,생산량,소비량
0,2017,2065.960,2090.518
1,2018,2251.992,2339.103
2,2019,2430.243,2430.139
3,2020,2528.167,2442.837
4,2021,2784.683,2962.181
5,2022,3089.143,3003.505
6,2023,3429.284,3171.547


In [23]:
import numpy as np

df_nickel['생산량'] = np.nan
df_nickel['소비량'] = np.nan

In [24]:
df_nickel

,기준일,기준가격,재고량,수급안정화지수,환율,생산량,소비량
0,2017-01-01,9984.29,374047,76.63,1169.209961,NaN,NaN
1,2017-02-01,10619.50,381745,78.24,1133.939941,NaN,NaN
2,2017-03-01,10230.43,381883,74.56,1117.020020,NaN,NaN
3,2017-04-01,9668.61,375819,77.10,1135.640015,NaN,NaN
4,2017-05-01,9154.29,380589,80.06,1123.900024,NaN,NaN
...,...,...,...,...,...,...,...
79,2023-08-01,20497.73,37074,28.00,1323.030029,NaN,NaN
80,2023-09-01,19629.05,39516,30.97,1351.930054,NaN,NaN
81,2023-10-01,18255.23,43572,31.98,1348.069946,NaN,NaN
82,2023-11-01,16979.77,43515,41.74,1288.359985,NaN,NaN


In [25]:
# 생산량과 소비량을 연도별로 월 단위로 나누어서 할당

for i, r in df_nickel.iterrows():
    year = r['기준일'].year
    
    for idx, row in production_consumption.iterrows():
        if row['연도']==year:
            df_nickel.iloc[i, 5] = round(row['생산량']/12, 2)
            df_nickel.iloc[i, 6] = round(row['소비량']/12, 2)

In [26]:
df_nickel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   기준일      84 non-null     datetime64[ns]
 1   기준가격     84 non-null     float64       
 2   재고량      84 non-null     int64         
 3   수급안정화지수  84 non-null     float64       
 4   환율       84 non-null     float64       
 5   생산량      84 non-null     float64       
 6   소비량      84 non-null     float64       
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 4.7 KB


In [39]:
df_nickel.describe()

,기준일,기준가격,재고량,수급안정화지수,환율,생산량,소비량
count,84,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000
mean,2020-06-16 01:08:34.285714176,16534.381786,193718.738095,39.705476,1187.010716,221.184286,219.522857
min,2017-01-01 00:00:00,8930.680000,37074.000000,6.240000,1059.900024,172.160000,174.210000
25%,2018-09-23 12:00:00,12477.230000,72929.250000,16.585000,1122.632477,187.670000,194.930000
50%,2020-06-16 00:00:00,15209.440000,206399.000000,38.860000,1168.534973,210.680000,203.570000
75%,2022-03-08 18:00:00,19990.800000,255798.500000,60.375000,1230.577515,257.430000,250.290000
max,2023-12-01 00:00:00,28853.750000,385273.000000,82.860000,1430.170044,285.770000,264.300000
std,NaN,5160.990069,114615.840014,24.056437,86.581274,37.233159,31.585517


In [29]:
import numpy as np
from scipy import stats
# 이상치 제거
# Z-score를 사용한 이상치 탐지
z_scores = np.abs(stats.zscore(df_nickel['기준가격']))
threshold = 2.5 # 임계값
z_outliers = (z_scores > threshold)

# 이상치 날짜 리스트 만들기
date_list = []
for i, r in df_nickel[z_outliers].iterrows():
    d = r[0]
    date_list.append(d)

# 해당 날짜 가격 nan으로 대체
for i, r in df_nickel.iterrows():
    if r['기준일'] in date_list:
        df_nickel.at[i, '기준가격'] = np.nan

# 결측치 보간
df_nickel['기준가격'] = df_nickel['기준가격'].interpolate(method='linear').round(2)

In [30]:
date_list

[Timestamp('2022-03-01 00:00:00'), Timestamp('2022-04-01 00:00:00')]

In [31]:
df_nickel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   기준일      84 non-null     datetime64[ns]
 1   기준가격     84 non-null     float64       
 2   재고량      84 non-null     int64         
 3   수급안정화지수  84 non-null     float64       
 4   환율       84 non-null     float64       
 5   생산량      84 non-null     float64       
 6   소비량      84 non-null     float64       
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 4.7 KB


In [32]:
df_nickel[df_nickel['기준일'].isin(date_list)]

,기준일,기준가격,재고량,수급안정화지수,환율,생산량,소비량
62,2022-03-01,25435.33,74771,6.24,1210.439941,257.43,250.29
63,2022-04-01,26692.67,73029,6.34,1263.380005,257.43,250.29


In [40]:
df_nickel.to_csv('nickel.csv',index=False)

In [41]:
production_consumption.to_csv('nickel_production_consumption.csv', index=False)